# Predviđanje ponašanja klijenata - tim Petty

Ova *Jupyter* bilježnica predstavlja objedinjenje svih ostalih *Jupyter* bilježnica i *Python* skripti na kojima je tim *Petty* radio tijekom natjecanja *Mozgalo* 2019. godine.

Tim *Petty*, svibanj 2019. godine.


In [1]:
# Stadardna Python biblioteka
import sys

# SciPy paketi
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Postavi prikaz grafova unutar biljeznice.
%matplotlib inline

# Postavi stil grafova na `ggplot'.
plt.style.use('ggplot')

# Ispisi inacice koristenog softvera.
print('Python version: {}'.format(sys.version))
print('Numpy version: {}'.format(np.__version__))
print('Pandas version: {}'.format(pd.__version__))
print('Matplotlib version: {}'.format(mpl.__version__))


Python version: 3.7.3 | packaged by conda-forge | (default, Mar 27 2019, 23:01:00) 
[GCC 7.3.0]
Numpy version: 1.16.3
Pandas version: 0.24.2
Matplotlib version: 3.0.3


##  Sadržaj:
***

1.  [Uvod](#uvod)
2.  [Problem](#problem)
3.  [Test podaci](#test-podatci)
    1.    [Učitavanje podataka](#ucitavanje-podataka)
    2.    [Eksploratorna analiza](#eksploratorna-analiza)
4.  [Obrada podataka](#obrada-podataka)
    1.    [Čišćenje podataka](#ciscenje-podataka)
    2.    [Spljoštenje](#spljostenje)
    3.    [*Feature engineering*](#feature-engineering)
    4.    [Trening i validacijski skup](#trening-i-validacijski-skup)
5.  [Model](#model)
    1.    [*CatBoost*](#catboost)
    2.    [Optimizacija parametara](#optimizacija-parametara)
    3.    [Treniranje](#treniranje)
    4.    [Testiranje](#testiranje)
6.  [Zaključak](#zakljucak)

***


##  Uvod <a class="anchor" id="uvod"></a>

U suvremeno doba vrlo je popularna ideja *prognoziranja* u širokom spektru djelatnosti, a posebnu pažnju dobiva u onim područjima u kojima neočekivani ishod može imati katastrofalne posljedice. Da bi prognoziranje bilo teorijski moguće, nužna je ovisnost i korelacija između poznatih vrijednosti i onih koje želimo predvidjeti; a, da bi prognoziranje bilo praktično moguće, u pravilu za konstrukciju prognostičkog modela, pa čak i za njegovo prognoziranje, potrebno je osimsliti efikasni računalni algoritam koji automatizira matematički račun na kojemu se model osniva.

Budući da je mjerenje i otkrivanje ovisnosti (a posebno korelacije) u domeni matematike i osmišljavanje algoritma u domeni računarstva, kao studenti diplomskog studija *Računarstvo i matematika* na *Prirodoslovno-matematičkog fakulteta* *Sveučilišta u Zagrebu* kolege **Luka Naglić**, **Davor Penzar** i **Domagoj Ravlić** objedinili su se u tim *Petty* kao natjecatelji u natjecanju *Mozgalo* &mdash; studentskom natjecanju čija je tema *data science*, ove godine konkretno binarna klasifikacija na temelju nekih poznatih podataka. Istovremeno, ovo rješenje natjecateljskog zadatka služi im i kao rješenje projektnog zadatka za kolegij *Strojno učenje* na njihovom studiju.


##  Problem <a class="anchor" id="problem"></a>

*Naručitelj* ovogodišnjeg zadatka za natjecanje *Mozgalo* je *Raiffeisenbank Hrvatska*, a kao zadatak postavili su predviđanje ponašanja svojih klijenata. Naime, problem je predvidjeti hoće li klijent prijevremeno raskinuti ugovor o kreditu odnosno depozitu &mdash; u slučaju kredita to bi značilo povrat cijelog duga prije isteka roka, a u slučaju depozita to bi značilo povlačenje svog uloženog novca prije dogovorenog datuma.


##  Podatci <a class="anchor" id="podatci"></a>

Sirovi podatci koji pripadaju takozvanom *trening-skupu* dani su tablicom u *.csv* formatu čiji svaki radak predstavlja izvještaj na kraju kvartala o točno jednom ugovoru. Svaki takav redak sadrži sljedeće informacije:

1.  *DATUM_IZVJESTAVANJA* &ndash; datum s kraja kvartala kada je promatrani izvještaj načinjen,
2.  *KLIJENT_ID* &ndash; jedinstvena numerička šifra klijenta,
3.  *OZNAKA_PARTIJE* &ndash; jedinstvena numerička šifra ugovora,
4.  *DATUM_OTVARANJA* &ndash; datum otvaranja ugovora,
5.  *PLANIRANI_DATUM_ZATVARANJA* &ndash; ugovoreni datum svršetka ugovora,
6.  *DATUM_ZATVARANJA* &ndash; stvarni datum svršetka ugovora,
7.  *UGOVORENI_IZNOS* &ndash; ugovoreni iznos u HRK,
8.  *STANJE_NA_KRAJU_PRETH_KVARTALA* &ndash; stanje ugovora (iznos u HRK) na kraju kvartala koji je prethodio kvartalu izvještaja,
9.  *STANJE_NA_KRAJU_KVARTALA* &ndash; stanje ugovora (iznos u HRK) na kraju kvartala izvještaja,
10. *VALUTA* &ndash; jedinstvena numerička šifra valute ugovora,
11. *VRSTA_KLIJENTA* &ndash; jedinstvena numerička šigra vrste klijenta,
12. *PROIZVOD* &ndash; jedinstvena šifra proizvoda,
13. *VRSTA_PROIZVODA* &ndash; jedinstvena šifra vrste proizvoda,
14. *VISINA_KAMATE* &ndash; kamatni koeficijent na kraju kvartala izvještaja,
15. *TIP_KAMATE* &ndash; jedinstvena šifra tipa kamate,
16. *STAROST* &ndash; starost klijenta na kraju kvartala izvještaja,
17. *PRIJEVREMENI_RASKID* &ndash; ciljna varijabla koja je istinita ako je ugovor raskinut prije ugovorenog datuma, a laž inače; **napomena:** dobivene vrijednosti treba korigirati [formulom](#eq:1).

Korekcija varijable *PRIJEVREMENI_RASKID*:
$$ \text{PRIJEVREMENI_RASKID} \left( x \right) \iff \text{DATUM_ZATVARANJA} \left( x \right) + 10 \, \text{dana} < \text{PLANIRANI_DATUM_ZATVARANJA} \left( x \right) \text{.} $$ <a class="anchor" id="eq:1"></a>


### Učitavanje podataka <a class="anchor" id="ucitavanje-podataka"></a>


Podatke ćemo iz datoteke *training_dataset_enc.csv* učitati kao `pandas.DataFrame` pozivom funkcije `pandas.read_pickle` odnosno `pandas.read_csv`. Pokretanjem sljedećih programskih linija može se primijetiti da učitavanje podataka iz tablice u *.csv* formatu traje dugo zato što redaka u tablici ima više od $ 5 $ milijuna, stoga se pri prvom otvaranju tablica odmah sprema i komprimirana kao *pickle* u *.pkl* formatu koji se u svakom konsekutivnom pozivu učitava znatno brže. Pri učitavanju iz *.csv* formata odmah ćemo:

1.  izbaciti stupac bez značenja (objasnio naručitelj zadatka; stupac se ni ne nalazi u prethodnom opisu skupa podataka) iz učitane tablice,
2.  korigirati varijablu *PRIJEVREMENI_RASKID*,
3.  retke tablice poredati leksikografski po ključu $ \left( \text{DATUM_IZVJESTAVANJA} , \text{DATUM_OTVARANJA} , \text{PLANIRANI_DATUM_ZATVARANJA} , \text{OZNAKA_PARTIJE} , \text{KLIJENT_ID} \right) $ pri čemu, totalnosti uređaja radi, za varijable *OZNAKA_PARTIJE* i *KLIJENT_ID* uzimamo vrijednosti koje njihove numeričke oznake predstavljaju u dekadskom zapisu,
4.  stupcima kategoričkog značenja (*KLIJENT_ID*, *OZNAKA_PARTIJE*, *VALUTA*, *VRSTA_KLIJENTA*, *PROIZVOD*, *VRSTA_PROIZVODA*, *TIP_KAMATE*) tip vrijednosti postavit ćemo na kategorički.


Podatke ćemo iz datoteke *training_dataset_enc.csv* učitati kao `pandas.DataFrame` pozivom funkcije `pandas.read_pickle` odnosno `pandas.read_csv`. Pokretanjem sljedećih programskih linija može se primijetiti da učitavanje podataka iz *CSV* datoteke traje jako dugo zato što redaka u tablici ima više od $ 5 $ milijuna, stoga ćemo prvo pokušati otvoriti *pickle*, a, ako datoteka koju tražimo ne postoji, tek ćemo onda otvarati *CSV* i odmah ćemo učitanu tablicu spremiti kao *pickle* datoteku za kasnije brže učitavanje. Izmjerit ćemo i vrijeme potrebno za učitavanje podataka jer se iz izmjerenih vremena jasno vidi vremenska optimizacija učitavanja podataka.

In [ ]:
df = None

try:
    df = pd.read_pickle('../data/training_dataset_enc.pkl')
except FileNotFoundError:
    df = pd.read_csv(
        '../data/training_dataset_enc.csv',
        index_col = 0,
        parse_dates = [
            'DATUM_IZVJESTAVANJA',
            'DATUM_OTVARANJA',
            'PLANIRANI_DATUM_ZATVARANJA',
            'DATUM_ZATVARANJA'
        ],
        infer_datetime_format = True,
        false_values = ['N'],
        true_values = ['Y']
    )
    df.drop(
        columns = ['Unnamed', 'Unnamed: 0', 'Unnamed: 0.1'],
        errors = 'ignore',
        inplace = True
    )
    df.reset_index(drop = True, inplace = True)
    df.PRIJEVREMENI_RASKID = (
        df.DATUM_ZATVARANJA + pd.Timedelta(10, 'D') <
        df.PLANIRANI_DATUM_ZATVARANJA
    )
    df.sort_values(
        by = [
            'DATUM_IZVJESTAVANJA',
            'DATUM_OTVARANJA',
            'PLANIRANI_DATUM_ZATVARANJA',
            'OZNAKA_PARTIJE',
            'KLIJENT_ID'
        ],
        ascending = True,
        inplace = True
    )
    for stupac in iter(
        [
            'KLIJENT_ID',
            'OZNAKA_PARTIJE',
            'VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'TIP_KAMATE'
        ]
    ):
        vrijednosti = df[stupac].dropna().unique()
        try:
            if isinstance(vrijednosti, np.ndarray):
                vrijednosti.sort()
            else:
                vrijednosti.sort_values(inplace = True)
        except (TypeError, ValueError, AttributeError):
            pass
        df[stupac] = df[stupac].astype(
            pd.api.types.CategoricalDtype(
                categories = vrijednosti,
                ordered = False
            )
        )
    df.reset_index(drop = True, inplace = True)
    df.to_pickle('../data/training_dataset_enc.pkl')


### Eksploratorna analiza <a class="anchor" id="eksploratorna-analiza"></a>

Prvo, pogledajmo početak tablice i informacije o tablici.

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.PRIJEVREMENI_RASKID.astype(float).sum() / df.shape[0]

Katastrofalni balans klasa trenutno je donekle i rezultat toga što tablica nije spljoštena &mdash; ako se neki ugovor raskinuo prijevremeno, u svim izvještajima prije raskida ugovora još uvijek će ciljna varijabla biti *laž*, iako se ti izvještaji zapravo odnose na ugovor koji je prijevremeno raskinut. Podatke ćemo spljoštiti u dijelu [Spljoštenje](#spljostenje).

Pogledajmo sada vremenske raspone kojima podatci pripadaju.


In [ ]:
pd.DataFrame(
    data = [
        [
            df.DATUM_IZVJESTAVANJA.min(),
            df.DATUM_OTVARANJA.min(),
            df.PLANIRANI_DATUM_ZATVARANJA.min(),
            df.DATUM_ZATVARANJA.min()
        ],
        [
            df.DATUM_IZVJESTAVANJA.max(),
            df.DATUM_OTVARANJA.max(),
            df.PLANIRANI_DATUM_ZATVARANJA.max(),
            df.DATUM_ZATVARANJA.max()
        ]
    ],
    columns = [
        'DATUM_IZVJESTAVANJA',
        'DATUM_OTVARANJA',
        'PLANIRANI_DATUM_ZATVARANJA',
        'DATUM_ZATVARANJA'
    ],
    index = ['min', 'max']
)


Za stupce *UGOVORENI_IZNOS*, *VISINA_KAMATE* i *STAROST* pogledajmo jednostavnu deskriptivnu analizu (količina definiranih vrijednosti, aritmetička sredina, standardna devijacija i karakteristična petorka).


In [ ]:
df[['UGOVORENI_IZNOS', 'VISINA_KAMATE', 'STAROST']].describe()

U ostatku eksploratorne analize podatke ćemo proučavati po nekom ugovoru, a ne po cijeloj tablici.

In [ ]:
po_klijentima_i_partijama = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])
klijenti_i_partije = np.array(
    [
        list(i)
            for i in po_klijentima_i_partijama.size().sort_values(
                ascending = False
            ).index
    ],
    dtype = int,
    order = 'C'
)


Vrste proizvoda govore o tome je li ugovor kredit ili depozit, a može poprimiti dvije vrijednosti: *A* ili *L*.


In [ ]:
df.VRSTA_PROIZVODA.dtype.categories

Tijekom našeg rješavanja zadatka zaključili smo da *A* najvjerojatnije predstavlja kredite, a *L* depozite. U svakom slučaju, promotrimo kako, na primjer, napreduje stanje na kraju kvartala za neki ugovor vrste proizvoda *A* i za neki ugovor vrste proizvoda *L*. Za najinformativnije rezultate, uzet ćemo ugovore o kojima imamo najviše zapisa (najraniji ključevi u nizu `klijenti_i_partije`).


In [ ]:
najveci_A = None
najveci_L = None
for i in iter(klijenti_i_partije):
    trenutni = po_klijentima_i_partijama.get_group(tuple(i))

    if najveci_A is None and trenutni.VRSTA_PROIZVODA.iloc[0] == 'A':
        najveci_A = trenutni
    elif najveci_L is None and trenutni.VRSTA_PROIZVODA.iloc[0] == 'L':
        najveci_L = trenutni

    if not (najveci_A is None or najveci_L is None):
        break


In [ ]:
fig = plt.figure()

ax_A = fig.add_subplot(121)
ax_L = fig.add_subplot(122)

ax_A.set_title('A')
ax_L.set_title('L')

ax_A.plot(najveci_A.DATUM_IZVJESTAVANJA, najveci_A.STANJE_NA_KRAJU_KVARTALA)
ax_L.plot(najveci_L.DATUM_IZVJESTAVANJA, najveci_L.STANJE_NA_KRAJU_KVARTALA)


Progresija stanja na kraju kvartala sugerira da vrsta proizvoda *A* predstavlja kredit (kredit se kroz vrijeme vraća pa se stanje smanjuje), a da proizvod *L* predstavlja depozit (stanje raste zbog novog ulaganja u isti ugovor i/ili zbog kamate). Zapravo, među glavnim argumentima po kojima smo zaključili koja šifra vrste proizvoda odgovara kojoj vrsti proizvoda i jesu bili grafovi stanja na kraju kvartala.


##  Obrada podataka <a class="anchor" id="obrada-podataka"></a>

# !!!!!!!!!!!!!!!! TU TREBA DAVOR  SKIPRTE ZA CISCENJE I SPLJOSTENJE I ONDA JOS FEATUER ENG NA KRAJU!!!!!!!!!!!!!!!!!!!!!

### Čišćenje podataka <a class="anchor" id="ciscenje-podataka"></a>


### Spljoštenje <a class="anchor" id="spljostenje"></a>

In [2]:
df = None

try:
    df = pd.read_pickle('../data/spljosteni.pkl')
    df_eval = pd.read_pickle('../data/eval/spljosteni.pkl')
except FileNotFoundError:
    df = pd.read_csv(
        '../data/spljosteni.csv',
        index_col = 0,
        parse_dates = [
            'DATUM_ZATVARANJA',
            'PRVI_DATUM_OTVARANJA',
            'ZADNJI_DATUM_OTVARANJA',
            'PRVI_PLANIRANI_DATUM_ZATVARANJA',
            'ZADNJI_PLANIRANI_DATUM_ZATVARANJA'
        ],
        infer_datetime_format = True,
    )
    df.reset_index(drop = True, inplace = True)
    for stupac in iter(
        [
            'KLIJENT_ID',
            'OZNAKA_PARTIJE',
            'PRVA_VALUTA',
            'ZADNJA_VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'PRVI_TIP_KAMATE',
            'ZADNJI_TIP_KAMATE'
        ]
    ):
        vrijednosti = df[stupac].dropna().unique()
        try:
            if isinstance(vrijednosti, np.ndarray):
                vrijednosti.sort()
            else:
                vrijednosti.sort_values(inplace = True)
        except (TypeError, ValueError, AttributeError):
            pass
        df[stupac] = df[stupac].astype(
            pd.api.types.CategoricalDtype(
                categories = vrijednosti,
                ordered = False
            )
        )
    df.reset_index(drop = True, inplace = True)
    df.to_pickle('../data/spljosteni.pkl')
    
    df_eval = pd.read_csv(
        '../data/eval/spljosteni.csv',
        index_col = 0,
        parse_dates = [
            'PRVI_DATUM_OTVARANJA',
            'ZADNJI_DATUM_OTVARANJA',
            'PRVI_PLANIRANI_DATUM_ZATVARANJA',
            'ZADNJI_PLANIRANI_DATUM_ZATVARANJA'
        ],
        infer_datetime_format = True,
    )
    df_eval.reset_index(drop = True, inplace = True)
    for stupac in iter(
        [
            'KLIJENT_ID',
            'OZNAKA_PARTIJE',
            'PRVA_VALUTA',
            'ZADNJA_VALUTA',
            'VRSTA_KLIJENTA',
            'PROIZVOD',
            'VRSTA_PROIZVODA',
            'PRVI_TIP_KAMATE',
            'ZADNJI_TIP_KAMATE'
        ]
    ):
        vrijednosti = df_eval[stupac].dropna().unique()
        try:
            if isinstance(vrijednosti, np.ndarray):
                vrijednosti.sort()
            else:
                vrijednosti.sort_values(inplace = True)
        except (TypeError, ValueError, AttributeError):
            pass
        df_eval[stupac] = df_eval[stupac].astype(
            pd.api.types.CategoricalDtype(
                categories = vrijednosti,
                ordered = False
            )
        )
    df_eval.reset_index(drop = True, inplace = True)
    df_eval.to_pickle('../data/eval/spljosteni.pkl')

##### TREBA LI NAM OVO? DA, NE, MOŽDA? 

In [ ]:
#df = df.loc[df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA.dt.year < 2019].copy()
#df.reset_index(drop = True, inplace = True)

In [ ]:
df.PRIJEVREMENI_RASKID.sum()/len(df)

### *Feature engineering* <a class="anchor" id="feature-engineering"></a>

##### DOMAGOJEVO POVEZIVANJE S EKONOMIJOM 

In [ ]:
from script.ubaci_ekonomiju import *


df = pd.read_pickle('../data/spljosteni.pkl')
df_eval = pd.read_pickle('../data/eval/spljosteni.pkl')
def domagoj_ekonomija(df):
    ekonomski_indikatori = pd.read_csv('../ekonomski-pokazatelji/ekonomski_indikatori.csv', index_col=0)
    print('Ubacivanje zapocinje!\n')
    ## trendovi
    df['TREND_bdp'] = df.apply(lambda x: trend(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'BDP (%)',ekonomski_indikatori), axis=1)
    df['TREND_inflacija'] = df.apply(lambda x: trend(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'Inflacija (%)', ekonomski_indikatori), axis=1)
    df['TREND_nezaposlenosti'] = df.apply(lambda x: trend(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'Stopa nezaposlenosti (%)', ekonomski_indikatori), axis=1)
    df['TREND_nafta'] = df.apply(lambda x: trend(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'Cijena nafte', ekonomski_indikatori), axis=1)
    print('Zavrseno "ubacivanje" trend-ova')

    ## std-ovi
    df['STD_bdp'] = df.apply(lambda x: std(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], 'BDP (%)',ekonomski_indikatori), axis=1)
    df['STD_inflacija'] = df.apply(lambda x: std(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], 'Inflacija (%)',ekonomski_indikatori), axis=1)
    df['STD_nezaposlenosti'] = df.apply(lambda x: std(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], 'Stopa nezaposlenosti (%)',ekonomski_indikatori), axis=1)
    df['STD_nafta'] = df.apply(lambda x: std(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], 'Cijena nafte',ekonomski_indikatori), axis=1)
    print('Zavrseno "ubacivanje" std-ova')

    ## udaljenost prosjeka od referentne vrijednosti
    df['PROSJEK_bdp'] = df.apply(lambda x: udaljenost_od_prosjeka(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'BDP (%)',ekonomski_indikatori), axis=1)
    df['PROSJEK_inflacija'] = df.apply(lambda x: udaljenost_od_prosjeka(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'Inflacija (%)',ekonomski_indikatori), axis=1)
    df['PROSJEK_nezaposlenosti'] = df.apply(lambda x: udaljenost_od_prosjeka(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'Stopa nezaposlenosti (%)',ekonomski_indikatori), axis=1)
    df['PROSJEK_nafta'] = df.apply(lambda x: udaljenost_od_prosjeka(x['ZADNJI_DATUM_OTVARANJA'], x['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'], x['VRSTA_PROIZVODA'], 'Cijena nafte',ekonomski_indikatori), axis=1)
    print('Zavrseno "ubacivanje" udaljenosti')
    return df

In [ ]:
df = domagoj_ekonomija(df)
df_eval = domagoj_ekonomija(df_eval)

df.to_pickle('../data/spljosteni_i_domagojeva_ekonomija.pkl')
df_eval.to_pickle('../data/eval/spljosteni_i_domagojeva_ekonomija.pkl')


In [3]:
df = pd.read_pickle('../data/spljosteni_i_domagojeva_ekonomija.pkl')
df_eval = pd.read_pickle('../data/eval/spljosteni_i_domagojeva_ekonomija.pkl')

##### STARO POVEZIVANJE S EKONOMIJOM 

In [ ]:

a = (pd.to_numeric(df_eval.ZADNJI_PLANIRANI_DATUM_ZATVARANJA - df_eval.ZADNJI_DATUM_OTVARANJA) <0)

[i for i, x in enumerate(a) if x]


In [ ]:
def stara_ekonomija(df):
    #df = df.loc[df.DATUM_OTVARANJA.dt.year >= 2000]
    #df = df.sample(1000)
    #df.reset_index(drop = True, inplace = True)

    #df = temp_df.copy()
   

    ''' 
    eko = pd.read_excel('../ekonomski-pokazatelji/ukupna.xls', index_col = 0, header = 0)

    x = eko.loc[df.PRVI_DATUM_OTVARANJA.dt.year].reset_index(drop = True)

    df = pd.concat([df, x], axis = 1)
    '''
    #df.DATUM_ZATVARANJA = df.DATUM_ZATVARANJA.astype(float)
    df['duljina'] = (df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA - df.ZADNJI_DATUM_OTVARANJA).dt.days/365.2475
    df['slozeni_kamatni'] = (( 1 + df.PRVA_VISINA_KAMATE)**(df.DULJINA)) * df.PRVI_UGOVORENI_IZNOS
    #df['LOG'] = np.log( df.ZADNJI_UGOVORENI_IZNOS)

    #df.PRVI_DATUM_OTVARANJA = pd.to_numeric(df.PRVI_DATUM_OTVARANJA)
    #df.ZADNJI_DATUM_OTVARANJA = pd.to_numeric(df.ZADNJI_DATUM_OTVARANJA)
    #df.PRVI_PLANIRANI_DATUM_ZATVARANJA = pd.to_numeric(df.PRVI_PLANIRANI_DATUM_ZATVARANJA)
    #df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA = pd.to_numeric(df.ZADNJI_PLANIRANI_DATUM_ZATVARANJA)
    return df

In [ ]:
df = pd.read_pickle('../data/spljosteni_i_domagojeva_ekonomija.pkl')
df_eval = pd.read_pickle('../data/eval/spljosteni_i_domagojeva_ekonomija.pkl')

df = stara_ekonomija(df)
df_eval = stara_ekonomija(df_eval)

##### DOMAGOJEVI FEATUERI

In [ ]:
def dodavanje(df):
    df['PRVI_DATUM_OTVARANJA'] = pd.to_numeric(df['PRVI_DATUM_OTVARANJA']) / 10e11
    df['PRVI_PLANIRANI_DATUM_ZATVARANJA'] = pd.to_numeric(df['PRVI_PLANIRANI_DATUM_ZATVARANJA']) / 10e11
    df['ZADNJI_DATUM_OTVARANJA'] = pd.to_numeric(df['ZADNJI_DATUM_OTVARANJA']) / 10e11
    df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'] = pd.to_numeric(df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA']) / 10e11

    df['promjena_tipa_kamate'] = (~(df['PRVI_TIP_KAMATE'] == df['ZADNJI_TIP_KAMATE'])).astype(int)
    df.drop('PRVI_TIP_KAMATE', axis=1, inplace=True)

    df['promjena_visine_kamate'] = df['ZADNJA_VISINA_KAMATE'] - df['PRVA_VISINA_KAMATE']
    df.drop('PRVA_VISINA_KAMATE', axis=1, inplace=True)

    df['promjena_ugovorenog_iznosa'] = df['ZADNJI_UGOVORENI_IZNOS'] - df['PRVI_UGOVORENI_IZNOS']
    df.drop('PRVI_UGOVORENI_IZNOS', axis=1, inplace=True)

    df['jednostavni_kamatni'] = (df['ZADNJI_UGOVORENI_IZNOS'] + (df['ZADNJI_UGOVORENI_IZNOS'] * df['ZADNJA_VISINA_KAMATE'] * df['zadnja_duljina_trajanja'])/100 ) / df['duljina']
    df['log_zadnji_ugovoreni_iznos'] = np.log(df['ZADNJI_UGOVORENI_IZNOS'])
    return df

In [ ]:
df = dodavanje(df)
df_eval = dodavanje(df_eval)

##### SPREMANJE

In [ ]:
df.to_pickle('../data/spljosteni_s_ekonomijom.pkl')
df_eval.to_pickle('../data/eval/spljosteni_s_ekonomijom.pkl')

###  Trening i validacijski skup <a class="anchor" id="trening-i-validacijski-skup"></a>


In [ ]:
df = pd.read_pickle('../data/spljosteni_s_ekonomijom.pkl')
df_eval = pd.read_pickle('../data/eval/spljosteni_s_ekonomijom.pkl')

In [ ]:
df.columns

In [ ]:
df_eval.columns

In [ ]:
test_portion = 0.3
tolerance_epsilon = 1e-4
train_df = df
test_df = df
test_N = int(np.round(test_portion * df.shape[0]))
while True:
    I = np.sort(np.random.choice(df.shape[0], size = test_N, replace = False))
    train_df = df.drop(index = df.index[I]).copy()
    test_df = df.iloc[I].copy()
    if np.abs(
        train_df.PRIJEVREMENI_RASKID.astype(float).sum() / train_df.shape[0] -
        test_df.PRIJEVREMENI_RASKID.astype(float).sum() / test_df.shape[0]
        ) < tolerance_epsilon:
        break
train_y = train_df.PRIJEVREMENI_RASKID
train_x = train_df.drop('PRIJEVREMENI_RASKID',axis = 1)
test_y = test_df.PRIJEVREMENI_RASKID
test_x = test_df.drop('PRIJEVREMENI_RASKID',axis = 1)

##### POPISIVANJE KATEGORIČKIH STUPACA 

In [ ]:
from pandas.core.arrays.categorical import Categorical as Categorical
categorical_columns = [
    #'KLIJENT_ID',
    #'OZNAKA_PARTIJE',
    'PRVA_VALUTA',
    'ZADNJA_VALUTA',
    'VRSTA_KLIJENTA',
    'PROIZVOD',
    'VRSTA_PROIZVODA',
    #'PRVI_TIP_KAMATE',
    'ZADNJI_TIP_KAMATE'
]
for column in iter(categorical_columns):
    values = df[column].dropna().unique()
    if isinstance(column, Categorical):
        values = np.asarray(values)
    try:
        if isinstance(values, np.ndarray):
            values.sort()
        else:
            values.sort_values(inplace = True)
    except (TypeError, ValueError, AttributeError):
        pass
    df[column] = df[column].astype(
        pd.api.types.CategoricalDtype(categories = values, ordered = False)
    )
cat_stupci = categorical_columns

##  Model <a class="anchor" id="model"></a>

### *CatBoost* <a class="anchor" id="catboost"></a>

malo opisati kaj je catboost: http://learningsys.org/nips17/assets/papers/paper_11.pdf <br>
https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb

In [ ]:
import catboost as cb
from catboost import *
print('CatBoost version: {}'.format(cb.__version__))

In [ ]:
def cirkus_kratica(model, X, y):
    '''
        Funkcija koja vraca TP, FP, TN, FN vrijednosti za podatke X, provedene kroz mrezu neural_net,
        i njihove labele y
    '''
    predicted = model.predict(data=test_x).astype(bool)
    y = y.astype(bool)

    return(
        (predicted & y).sum(),
        (predicted & ~y).sum(),
        (~predicted & ~y).sum(),
        (~predicted & y).sum()
    )

def acc_f1(model, X, y):
    '''
        Funkcija koja racuna accuracy i F1-score za podatke X provedene kroz mrezu neural_net,
        i njihove labele y

        Povratni argument je tuple te dvije vrijednosti
    '''
    
    tp, fp, tn, fn = cirkus_kratica(model, X, y)
    # print('\nTP: {}, FP: {}, TN: {}, FN: {}'.format(tp,fp,tn,fn))

    precision = (tp/(tp+fp)) if (tp+fp) else 0.0
    recall = (tp/(tp+fn)) if (tp+fn) else 0.0

    accuracy = (tp+tn)/(tp + fp + tn + fn)
    f1 = 2*precision*recall/(precision+recall) if (precision+recall) else 0

    return (accuracy, f1, precision, recall)

def daj_tezine (model, train_data):
    return pd.DataFrame(model.get_feature_importance(), index = train_data.columns, columns = ['Tezine']).sort_values(by = 'Tezine', ascending = False)


### Optimizacija parametara <a class="anchor" id="optimizacija-parametara"></a>

sljedeći kod očekuje:
train_x, train_x, train_y, test_y, cat_stupci

In [ ]:
from sklearn.metrics import mean_absolute_error
from hyperopt import hp, tpe, STATUS_OK, STATUS_FAIL, Trials, fmin


def obj(space):
    if(space['iterations'] < 1 or space['learning_rate'] < 0):
        return {'status': STATUS_FAIL }
    if(space['border_count'] < 1 or space['border_count'] >254 ):
        return {'status': STATUS_FAIL }
    model = CatBoostClassifier(
        learning_rate=space['learning_rate'],
        iterations=space['iterations'],
        l2_leaf_reg = space['l2_leaf_reg'],
        depth = round(space['depth']),
        random_seed=space['random_seed'],
        border_count=space['border_count'],
        
        bagging_temperature = space['bagging_temperature'],
        sampling_unit = space['sampling_unit'],
        random_strength = space['random_strength'],
        #grow_policy = space['grow_policy'], # ovo ide samo ako je GPU
        leaf_estimation_method = space['leaf_estimation_method'],
        #score_function = space['score_function'], # ovo ide samo ako je GPU
        od_type = space['od_type'],
        
        class_weights= [1,len(train_y)/train_y.sum()],
        custom_loss=['F1', 'Accuracy'],
        task_type ='GPU'

    )
    
    
    model.fit(
        train_x, train_y,
        eval_set=(test_x, test_y),
        cat_features=cat_stupci,
        verbose=False

    )
    
    '''
    #ovo je za cross validaciju
    cv_data = cb.cv(
        Pool(df.drop('PRIJEVREMENI_RASKID',axis = 1), df.PRIJEVREMENI_RASKID, cat_features=cat_stupci),
        model.get_params()
    )
    acc = np.max(cv_data['test-Accuracy-mean'])
    '''
    #pred = model.predict(test_x)
    #mae = mean_absolute_error((test_y), (pred))
    
    acc, f1, _, _ = acc_f1(model,test_x, test_y)
    
    #tu možda vraćati umjesto mae, 1- accuracy?
    #return{'loss':mae, 'status': STATUS_OK }
    return{'loss':1 - acc, 'status': STATUS_OK }

# https://github.com/hyperopt/hyperopt/wiki/FMin
# gore su ostale funkcije hp.uniform, hp.randint ... za space

#https://catboost.ai/docs/concepts/parameter-tuning.html
#gore je koji sve parametri bi se trebali tunat
space ={
        'random_seed': hp.randint('random_seed',1000),
        'l2_leaf_reg': hp.qlognormal('l2_leaf_reg', 1, 5, 0.5),
        'learning_rate': hp.uniform('learning_rate', 1e-4,5e-1),
        'iterations': hp.randint ('iterations', 600),
        'depth': hp.quniform ('depth', 4,10, 0.5),
        'bagging_temperature' : hp.lognormal('bagging_temperature', 1,1),
        'random_strength' : hp.lognormal('random_strength', 1,1),
        'border_count': hp.randint('border_count', 254), 
    
        'sampling_unit' : hp.choice('sampling_unit', ['Group', 'Object']),
        #'grow_policy' : hp.choice('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']), #za GPU; nema feat importance
        'leaf_estimation_method' : hp.choice('leaf_estimation_method', ['Newton', 'Gradient']),
        #'score_function' : hp.choice('score_function', ['Correlation', 'L2', 'NewtonCorrelation', 'NewtonL2']), #za GPU
        'od_type' : hp.choice('od_type', ['IncToDec', 'Iter'])
    }


trials = Trials()
best = fmin(fn=obj,
            space=space,
            algo=tpe.suggest,
            max_evals=20, # change
            trials=trials
        )

print(best)

In [ ]:
trials.vals

trials.trials

### Treniranje <a class="anchor" id="treniranje"></a>


In [ ]:
(1-train_y).sum()/train_y.sum(), len(train_y)/train_y.sum()

In [ ]:
sampling_unit_ = ['Group', 'Object']
grow_policy_ = ['SymmetricTree', 'Depthwise', 'Lossguide']
leaf_estimation_method_ = ['Newton', 'Gradient']
score_function_ = ['Correlation', 'L2', 'NewtonCorrelation', 'NewtonL2']
od_type_ = ['IncToDec', 'Iter']
#očekuje best varijablu koja je gore izračunata uz ostale varijable
t0 = time.time()
model = CatBoostClassifier(
    learning_rate=1.2,
    iterations=best['iterations'],
    l2_leaf_reg = best['l2_leaf_reg'],
    depth = round(best['depth']),
    random_strength = best['random_strength'],
    bagging_temperature = best['bagging_temperature'],
    border_count = best['border_count'],
    
    #train_dir='nas_model',
    
    sampling_unit = sampling_unit_[best['sampling_unit']],    
    #grow_policy = grow_policy_[best['grow_policy']], #za GPU
    leaf_estimation_method = leaf_estimation_method_[best['leaf_estimation_method']],
    #score_function = score_function_[best['score_function']], #za GPU
    od_type = od_type_[best['od_type']],

    random_seed=best['random_seed'],
    class_weights=[1,len(train_y)/train_y.sum()],
    custom_loss=['F1', 'Accuracy'],
    task_type ='CPU'

)
model.fit(
    train_x, train_y,
    eval_set=(test_x, test_y),
    cat_features=cat_stupci,
    verbose=False,
    plot = True
)
t1 = time.time()

# Izracunaj vremenski period od t0 do t1 u sekundama.
d = float(t1 - t0)

# Ispisi vrijeme.
print(
    'Trajanje ucitavanja: {h:d}h {m:02d}m {s:06.3f}s ({S:.3f}s)'.format(
        h = int(math.floor(d / 3600)),
        m = int(math.floor((d - 3600 * int(math.floor(d / 3600))) / 60)),
        s = d - 60 * int(math.floor(d / 60)),
        S = d
    )
)

# Oslobodi memoriju.
del d
del t0
del t1    



In [ ]:
#model.save_model('nas_model1')

In [ ]:
acc_f1(model,test_x, test_y)

In [ ]:
pd.DataFrame(model.get_feature_importance(), index = test_x.columns).sort_values(by=0,ascending=False)

In [ ]:
daj_tezine(model, train_x)

In [ ]:
cv_params = model.get_params()
cv_data = cb.cv(
    Pool(df.drop('PRIJEVREMENI_RASKID',axis = 1), df.PRIJEVREMENI_RASKID, cat_features=cat_stupci),
    cv_params,
    plot=True
)

In [ ]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))


In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

### Testiranje <a class="anchor" id="testiranje"></a>


In [ ]:
df = pd.read_pickle('../data/eval/spljosteni_sa_ekonomijom.pkl')
df.set_index('instance_id', inplace=True)

df.drop(columns=['KLIJENT_ID', 'OZNAKA_PARTIJE', 'STAROST', 'PRIJEVREMENI_RASKID'], axis = 1, inplace = True)
# df = df[(df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.year < 2019) | ((df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.year == 2019) & (df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.month == 1) & (df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.day <= 10))]


## pocisti ZADNJI_UGOVORENI_IZNOS == 0 jer to ne ulazi u evaluaciju na internetu
df = df[df['ZADNJI_UGOVORENI_IZNOS'] != 0]

In [ ]:
df['zadnja_duljina_trajanja'] = (df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'] - df['ZADNJI_DATUM_OTVARANJA']).dt.days

df['PRVI_DATUM_OTVARANJA'] = pd.to_numeric(df['PRVI_DATUM_OTVARANJA']) / 10e11
df['PRVI_PLANIRANI_DATUM_ZATVARANJA'] = pd.to_numeric(df['PRVI_PLANIRANI_DATUM_ZATVARANJA']) / 10e11
df['ZADNJI_DATUM_OTVARANJA'] = pd.to_numeric(df['ZADNJI_DATUM_OTVARANJA']) / 10e11
df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'] = pd.to_numeric(df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA']) / 10e11


In [ ]:
predicted = df[df['PRIJEVREMENI_RASKID'] == 1]
predicted = predicted.loc[:, df.columns.intersection(['PRIJEVREMENI_RASKID'])]

In [ ]:
predicted = predicted.loc[:, df.columns.intersection(['PRIJEVREMENI_RASKID'])]

In [ ]:
predicted = predicted.loc[:, df.columns.intersection(['PRIJEVREMENI_RASKID'])]

In [ ]:
predicted = predicted.loc[:, df.columns.intersection(['PRIJEVREMENI_RASKID'])]

In [ ]:
original['PRIJEVREMENI_RASKID'] = original['PRIJEVREMENI_RASKID'].map({1: 'Y', 0: 'N'})


In [ ]:
original.to_csv('../data/student.csv')

##  Zaključak <a class="anchor" id="zakljucak"></a>